# Rough Outline

My objective was to simply compare how tools used and education level varied across different job roles.


First , I get an idea of the variables ,like how many nulls are there, and so on. Then I try to build graphs to compare across job roles.  

[Reference notebook](https://www.kaggle.com/kenjee/kaggle-project-from-scratch)

# Importing Packages

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data=pd.read_csv("/kaggle/input/kaggle-survey-2021/kaggle_survey_2021_responses.csv")
pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns',500)
# data.shape
# data.head(10)
# data.describe()
data.info(verbose = False)
data.head()

# Data Cleaning and Barplots

This section deals with cleaning the data and making simple plots to see how the responses are distributed

In [ ]:
df_final=data.iloc[1:,:]
df_final.head()

In [ ]:
df_final.isnull().sum()/df_final.shape[0]

In [ ]:
# I wanted to make the data useful by breaking it down to individual questions. The most pragmatic way was to create a dictionary .
Questions={}

qnums = list(dict.fromkeys([i.split('_')[0] for i in df_final.columns]))
qnums

In [ ]:
for i in qnums:
    if i in ['Q1','Q2','Q3']:
        Questions[i]=df_final[i]
    else :
        Questions[i]=df_final[[q for q in df_final.columns if q.startswith(i)]]

In [ ]:
df_final['Q1']
df_final_sorted_age=df_final.sort_values('Q1')

In [ ]:
fig=px.histogram(df_final_sorted_age,x='Q1',
                 labels={'Q1': "Age",
                    },
                title="Age in ascending order")


fig.update_layout(
    font_family='Carousel',
    font_color='Maroon',
    title_font_family='Carousel',
    title_font_color = 'Maroon'
)
fig.update_xaxes(title_font_family="Arial")

fig.show()

In [ ]:
# seeing what columns are contained in questions regarding coding experience and age

print(df_final.Q6.unique())
print(df_final.Q1.unique())

In [ ]:
fig2=px.density_heatmap(df_final, x='Q1', y='Q6', nbinsx=15, nbinsy=15, color_continuous_scale="Viridis",
                        category_orders={'Q1':['18-21','22-24','25-29','30-34','35-39','40-44','45-49','50-54','55-59','60-69','70+'],
                                        'Q6' :['I have never written code','< 1 years', '1-3 years','3-5 years','5-10 years','10-20 years','20+ years']},
                       labels={"Q1" : "Age" , "Q2" : "Experience"},
                       title="Correlation between Age and Experience ")


fig2.update_layout(
    font_family='Carousel',
    font_color='Brown',
    title_font_family='Carousel',
    title_font_color = 'Seagreen'
)
fig2.show()

In [ ]:
Questions['Q7']

In [ ]:
Questions["Q7"].columns =list(Questions['Q7'].mode().iloc[0,:])
q7=Questions['Q7'].count().reset_index()
q7.columns=['language','Count']
q7=q7.sort_values('Count',ascending = False)
fig=go.Figure([go.Bar(x=q7.language, y= q7.Count)]) 
fig.show()

In [ ]:
Roles={}
for i in df_final.Q5.unique():
    Roles[i] = df_final[df_final.Q5 == i]

In [ ]:
Roles.keys()

In [ ]:
Roles['Student']

In [ ]:
edu=df_final.Q4.value_counts()
edu

In [ ]:
fig=go.Figure([go.Bar(x=edu.index, y=edu.values)])
fig.show()

# Building Advanced Graphs

Comparing how education levels vary across roles, or how preferred programming languages or IDE vary across job roles.

In [ ]:
fig=go.Figure(layout=go.Layout(title= go.layout.Title(text="Comparing Education by Position")))

fig.add_trace(go.Bar(name='Role Selection', x=edu.index,y=(edu.values/edu.values.sum())))
fig.add_trace(go.Bar(name='All Data', x=edu.index,y=(edu.values/edu.values.sum())))


buttons=[]

for i in list(Roles.keys())[1:]:
    buttons.append(dict(method='restyle',
                       label=i,
                       visible=True,
                       args=[{'y' : [Roles[i].Q4.value_counts().values/Roles[i].Q4.value_counts().values.sum()],
                              'x' : [Roles[i].Q4.value_counts().index],
                             'type':'bar'},[0]],
                       )
                  )

updatemenu=[]
your_menu={}
updatemenu.append(your_menu)

updatemenu[0]['buttons'] = buttons
updatemenu[0]['direction'] = 'down'
updatemenu[0]['showactive'] = True

fig.update_layout(updatemenus=updatemenu)

fig.update_xaxes(categoryorder= 'array', categoryarray= ["Doctoral degree",'Master’s degree',
'Bachelor’s degree','Some college/university study without earning a bachelor’s degree',"Professional degree",
"No formal education past high school","I prefer not to answer"])

fig.show()

In [ ]:
fig=go.Figure(layout=go.Layout(title= go.layout.Title(text="Comparing Education by Position")))
fig.add_trace(go.Bar(name='Role Selection',x=edu.index,y=(edu.values/edu.values.sum())))

buttons=[]

for i in list(Roles.keys())[1:]:
    buttons.append(dict(method='restyle',
                       label=i,
                       visible=True,
                       args=[{'y':[Roles[i].Q4.value_counts().values/Roles[i].Q4.value_counts().values.sum()],
                              'x':[Roles[i].Q4.value_counts().index],
                              'type':'bar'},[0]],
                       )
                  )
fig.add_trace(go.Bar(name='All Data', x =edu.index, y =(edu.values/edu.values.sum())))

buttons2 = []


for i in list(Roles.keys())[1:]:
    buttons2.append(dict(method='restyle',
                        label=i,
                        visible=True,
                        args=[{'y':[Roles[i].Q4.value_counts().values/Roles[i].Q4.value_counts().values.sum()],
                               'x':[Roles[i].Q4.value_counts().index],
                               'type':'bar'},[1]],
                        )
                   )

button_layer_1_height = 1.23
updatemenus = list([
    dict(buttons=buttons,
        direction='down',
        pad={'r':10,'t':10},
        showactive=True,
        x=0.1,
        xanchor='left',
        y=button_layer_1_height,
        yanchor='top'),
    dict(buttons=buttons2,
        direction='down',
        pad={'r':10,'t':10},
        showactive=True,
        x=0.5,
        xanchor='left',
        y=button_layer_1_height,
        yanchor='top')])


fig.update_layout( updatemenus=updatemenus)
fig.update_xaxes(categoryorder= 'array', categoryarray= ["Doctoral degree",'Master’s degree','Bachelor’s degree','Some college/university study without earning a bachelor’s degree',"Professional degree","No formal education past high school","I prefer not to answer"])
fig.show()


In [ ]:


Questions['Q7']['Roles'] = df_final.Q5

fig = go.Figure(layout=go.Layout(title= go.layout.Title(text="Comparing Coding Languages by Position")))

fig.add_trace(go.Bar(name= 'Selection 1', x= q7.language, y=(q7.Count/ q7.Count.sum())))

def filter_bars(role, data):
    df = data[data['Roles'] == role]
    q7 = df.drop('Roles', axis= 1).count().reset_index()
    q7.columns = ['language','Count']
    return (q7.language, q7.Count/q7.Count.sum())

buttons = []

#added button for all data comparison
buttons.append(dict(method='restyle',
                        label= 'All Samples',
                        visible=True,
                        args=[{'y':[(q7.Count/ q7.Count.sum())],
                               'x':[q7.language],
                               'type':'bar'}, [0]],
                        )
                  )

for i in list(Roles.keys())[1:]:
    buttons.append(dict(method='restyle',
                        label= i,
                        visible=True,
                        args=[{'y':[filter_bars(i,Questions['Q7'])[1].values],
                               'x':[filter_bars(i,Questions['Q7'])[0].values],
                               'type':'bar'}, [0]],
                        )
                  )

fig.add_trace(go.Bar(name= 'Selection 2', x= q7.language, y=(q7.Count/ q7.Count.sum())))

buttons2 = []
#added button for all data comparison
buttons2.append(dict(method='restyle',
                        label= 'All Samples',
                        visible=True,
                        args=[{'y':[(q7.Count/ q7.Count.sum())],
                               'x':[q7.language],
                               'type':'bar'}, [1]],
                        )
                  )

for j in list(Roles.keys())[1:]:
    buttons2.append(dict(method='restyle',
                        label= j,
                        visible=True,
                        args=[{'y':[filter_bars(j,Questions['Q7'])[1].values],
                               'x':[filter_bars(j,Questions['Q7'])[0].values],
                               'type':'bar'}, [1]],
                        )                        
                  )

# adjusted dropdown placement 
#found out updatemenus take a dictionary of buttons and allow you to format how the dropdowns look etc.
# https://plotly.com/python/dropdowns/
button_layer_1_height = 1.15
updatemenus = list([
    dict(buttons=buttons,
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.1,
            xanchor="left",
            y=button_layer_1_height,
            yanchor="top"),
    dict(buttons=buttons2,
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.50,
            xanchor="left",
            y=button_layer_1_height,
            yanchor="top")])
    
fig.update_layout( updatemenus=updatemenus)
#added annotations next to dropdowns 
fig.update_layout(
    annotations=[
        dict(text="Selection 1", x=0, xref="paper", y=1.1, yref="paper",
                             align="left", showarrow=False),
        dict(text="Selection 2", x=0.45, xref="paper", y=1.1,
                             yref="paper", showarrow=False)
    ])
fig.update_xaxes(categoryorder= 'array', categoryarray= q7.language)
fig.show()


In [ ]:

Questions['Q9'].columns = list(Questions['Q9'].mode().iloc[0,:])
q9 = Questions['Q9'].count().reset_index()
q9.columns = ['language','Count']
q9 = q9.sort_values('Count', ascending = False)

Questions['Q9']['Roles'] = df_final.Q5

fig = go.Figure(layout=go.Layout(title= go.layout.Title(text="Comparing IDE's by Position")))

fig.add_trace(go.Bar(name= 'Selection 1', x= q9.language, y=(q9.Count/ q9.Count.sum())))

buttons = []

#added button for all data comparison
buttons.append(dict(method='restyle',
                        label= 'All Samples',
                        visible=True,
                        args=[{'y':[(q9.Count/ q9.Count.sum())],
                               'x':[q9.language],
                               'type':'bar'}, [0]], # the [0] at the end lets us know they are for the first trace
                        )
                  )

for i in list(Roles.keys())[1:]:
    buttons.append(dict(method='restyle',
                        label= i,
                        visible=True,
                        args=[{'y':[filter_bars(i,Questions['Q9'])[1].values],
                               'x':[filter_bars(i,Questions['Q9'])[0].values],
                               'type':'bar'}, [0]], # the [0] at the end lets us know they are for the first trace
                        )
                  )

fig.add_trace(go.Bar(name= 'Selection 2', x= q9.language, y=(q9.Count/ q9.Count.sum())))

buttons2 = []
#added button for all data comparison
buttons2.append(dict(method='restyle',
                        label= 'All Samples',
                        visible=True,
                        args=[{'y':[(q9.Count/ q9.Count.sum())],
                               'x':[q9.language],
                               'type':'bar'}, [1]], # the [0] at the end lets us know they are for the first trace
                        )
                  )

for j in list(Roles.keys())[1:]:
    buttons2.append(dict(method='restyle',
                        label= j,
                        visible=True,
                        args=[{'y':[filter_bars(j,Questions['Q9'])[1].values],
                               'x':[filter_bars(j,Questions['Q9'])[0].values],
                               'type':'bar'}, [1]], # the [1] at the end lets us know they are for the first trace
                        )                        #literally figured that out by just experimenting 
                  )

# adjusted dropdown placement 
#found out updatemenus take a dictionary of buttons and allow you to format how the dropdowns look etc.
# https://plotly.com/python/dropdowns/
button_layer_1_height = 1.15
updatemenus = list([
    dict(buttons=buttons,
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.1,
            xanchor="left",
            y=button_layer_1_height,
            yanchor="top"),
    dict(buttons=buttons2,
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.50,
            xanchor="left",
            y=button_layer_1_height,
            yanchor="top")])
    
fig.update_layout( updatemenus=updatemenus)
#added annotations next to dropdowns 
fig.update_layout(
    annotations=[
        dict(text="Selection 1", x=0, xref="paper", y=1.1, yref="paper",
                             align="left", showarrow=False),
        dict(text="Selection 2", x=0.45, xref="paper", y=1.1,
                             yref="paper", showarrow=False)
    ])
fig.update_xaxes(categoryorder= 'array', categoryarray= q9.language)
fig.show()


In [ ]:
edu2 = df_final.Q8.value_counts()
fig = go.Figure(layout=go.Layout(title= go.layout.Title(text="Recommended Coding Languages by Position")))
fig.add_trace(go.Bar(name= 'Selection 1', x= edu2.index, y=(edu2.values/ edu2.values.sum())))

buttons = []


buttons.append(dict(method='restyle',
                        label= 'All Samples',
                        visible=True,
                        args=[{'y':[df_final.Q8.value_counts().values/df_final.Q8.value_counts().values.sum()],
                               'x':[df_final.Q8.value_counts().index],
                               'type':'bar'}, [0]], # the [0] at the end lets us know they are for the first trace
                        )
                  )

for i in list(Roles.keys())[1:]:
    buttons.append(dict(method='restyle',
                        label= i,
                        visible=True,
                        args=[{'y':[Roles[i].Q8.value_counts().values/Roles[i].Q8.value_counts().values.sum()],
                               'x':[Roles[i].Q8.value_counts().index],
                               'type':'bar'}, [0]], # the [0] at the end lets us know they are for the first trace
                        )
                  )

fig.add_trace(go.Bar(name= 'Selection 2',x= edu2.index, y=(edu2.values/ edu2.values.sum())))

buttons2 = []
buttons2.append(dict(method='restyle',
                        label= 'All Samples',
                        visible=True,
                        args=[{'y':[df_final.Q8.value_counts().values/df_final.Q8.value_counts().values.sum()],
                               'x':[df_final.Q8.value_counts().index],
                               'type':'bar'}, [1]], # the [0] at the end lets us know they are for the first trace
                        )
                  )

for i in list(Roles.keys())[1:]:
    buttons2.append(dict(method='restyle',
                        label= i,
                        visible=True,
                        args=[{'y':[Roles[i].Q8.value_counts().values/Roles[i].Q8.value_counts().values.sum()],
                               'x':[Roles[i].Q8.value_counts().index],
                               'type':'bar'}, [1]],
                        )                        
                  )

# adjusted dropdown placement 
#found out updatemenus take a dictionary of buttons and allow you to format how the dropdowns look etc.
# https://plotly.com/python/dropdowns/
button_layer_1_height = 1.15
updatemenus = list([
    dict(buttons=buttons,
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.1,
            xanchor="left",
            y=button_layer_1_height,
            yanchor="top"),
    dict(buttons=buttons2,
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.50,
            xanchor="left",
            y=button_layer_1_height,
            yanchor="top")])
    
fig.update_layout( updatemenus=updatemenus)
#added annotations next to dropdowns 
fig.update_layout(
    annotations=[
        dict(text="Selection 1", x=0, xref="paper", y=1.1, yref="paper",
                             align="left", showarrow=False),
        dict(text="Selection 2", x=0.45, xref="paper", y=1.1,
                             yref="paper", showarrow=False)
    ])
fig.show()

In [ ]:


def filter_bars(role, data):
    df = data[data['Roles'] == role]
    q = df.drop('Roles', axis= 1).count().reset_index()
    q.columns = ['language','Count']
    return (q.language, q.Count/q.Count.sum())

def build_graph(q_number, Roles, Title):
    """Create dropdown visual with question data"""
    if isinstance(q_number, pd.DataFrame):
        qnumber = q_number.copy()
        qnumber.columns = list(qnumber.mode().iloc[0,:])
        qcnt = qnumber.count().reset_index()
        qcnt.columns = ['feature','cnt']
        qcnt = qcnt.sort_values('cnt', ascending = False)
        qnumber['Roles'] = df_final.Q5
        

        fig = go.Figure(layout=go.Layout(title= go.layout.Title(text=Title)))
        
        fig.add_trace(go.Bar(name= 'Selection 1', x= qcnt.feature, y=(qcnt.cnt/ qcnt.cnt.sum())))

        buttons = []

        
        buttons.append(dict(method='restyle',
                                label= 'All Samples',
                                visible=True,
                                args=[{'y':[(qcnt.cnt/ qcnt.cnt.sum())],
                                       'x':[qcnt.feature],
                                       'type':'bar'}, [0]], 
                                )
                          )

        for i in list(Roles.keys())[1:]:
            buttons.append(dict(method='restyle',
                                label= i,
                                visible=True,
                                args=[{'y':[filter_bars(i,qnumber)[1].values],
                                       'x':[filter_bars(i,qnumber)[0].values],
                                       'type':'bar'}, [0]],
                                )
                          )

        fig.add_trace(go.Bar(name= 'Selection 2', x= qcnt.feature, y=(qcnt.cnt/ qcnt.cnt.sum())))

        buttons2 = []
        
        buttons2.append(dict(method='restyle',
                                label= 'All Samples',
                                visible=True,
                                args=[{'y':[(qcnt.cnt/ qcnt.cnt.sum())],
                                       'x':[qcnt.feature],
                                       'type':'bar'}, [1]], 
                                )
                          )

        for i in list(Roles.keys())[1:]:
            buttons2.append(dict(method='restyle',
                                label= i,
                                visible=True,
                                args=[{'y':[filter_bars(i,qnumber)[1].values],
                                       'x':[filter_bars(i,qnumber)[0].values],
                                       'type':'bar'}, [1]],
                                )
                          )

        
        button_layer_1_height = 1.15
        updatemenus = list([
            dict(buttons=buttons,
                    direction="down",
                    pad={"r": 10, "t": 10},
                    showactive=True,
                    x=0.1,
                    xanchor="left",
                    y=button_layer_1_height,
                    yanchor="top"),
            dict(buttons=buttons2,
                    direction="down",
                    pad={"r": 10, "t": 10},
                    showactive=True,
                    x=0.50,
                    xanchor="left",
                    y=button_layer_1_height,
                    yanchor="top")])

        fig.update_layout( updatemenus=updatemenus)
        
        fig.update_layout(
            annotations=[
                dict(text="Selection 1", x=0, xref="paper", y=1.1, yref="paper",
                                     align="left", showarrow=False),
                dict(text="Selection 2", x=0.45, xref="paper", y=1.1,
                                     yref="paper", showarrow=False)
            ])
        fig.update_xaxes(categoryorder= 'array', categoryarray= qcnt.feature)
        fig.show()
        
        
    else:
        qnumber= q_number.copy()
        vcnts = qnumber.value_counts()
        qnumber = pd.concat([qnumber,df_final.Q5], axis =1)
        qnumber.columns = ['feature','Roles']

        fig = go.Figure(layout=go.Layout(title= go.layout.Title(text=Title)))
        
        fig.add_trace(go.Bar(name= 'Selection 1', x= vcnts.index, y=(vcnts.values/ vcnts.values.sum())))

        buttons = []

        #added button for all data comparison
        buttons.append(dict(method='restyle',
                                label= 'All Samples',
                                visible=True,
                                args=[{'y':[vcnts.values/ vcnts.values.sum()],
                                       'x':[vcnts.index],
                                       'type':'bar'}, [0]], # the [0] at the end lets us know they are for the first trace
                                )
                          )

        for i in list(Roles.keys())[1:]:
            qrole = qnumber[qnumber['Roles']==i].feature.value_counts()
            buttons.append(dict(method='restyle',
                                label= i,
                                visible=True,
                                args=[{'y':[qrole.values/qrole.values.sum()],
                                       'x':[qrole.index],
                                       'type':'bar'}, [0]], # the [0] at the end lets us know they are for the first trace
                                )
                          )

        fig.add_trace(go.Bar(name= 'Selection 2',x= vcnts.index, y=(vcnts.values/ vcnts.values.sum())))

        buttons2 = []
        
        buttons2.append(dict(method='restyle',
                                label= 'All Samples',
                                visible=True,
                                args=[{'y':[(vcnts.values/ vcnts.values.sum())],
                                       'x':[vcnts.index],
                                       'type':'bar'}, [1]], # the [0] at the end lets us know they are for the first trace
                                )
                          )

        for i in list(Roles.keys())[1:]:
            qrole = qnumber[qnumber['Roles']==i].feature.value_counts()
            buttons2.append(dict(method='restyle',
                                label= i,
                                visible=True,
                                args=[{'y':[qrole.values/qrole.values.sum()],
                                       'x':[qrole.index],
                                       'type':'bar'}, [1]], # the [0] at the end lets us know they are for the first trace
                                )
                          )
        # adjusted dropdown placement 
        #found out updatemenus take a dictionary of buttons and allow you to format how the dropdowns look etc.
        # https://plotly.com/python/dropdowns/
        button_layer_1_height = 1.15
        updatemenus = list([
            dict(buttons=buttons,
                    direction="down",
                    pad={"r": 10, "t": 10},
                    showactive=True,
                    x=0.1,
                    xanchor="left",
                    y=button_layer_1_height,
                    yanchor="top"),
            dict(buttons=buttons2,
                    direction="down",
                    pad={"r": 10, "t": 10},
                    showactive=True,
                    x=0.50,
                    xanchor="left",
                    y=button_layer_1_height,
                    yanchor="top")])

        fig.update_layout( updatemenus=updatemenus)
        #added annotations next to dropdowns 
        fig.update_layout(
            annotations=[
                dict(text="Selection 1", x=0, xref="paper", y=1.1, yref="paper",
                                     align="left", showarrow=False),
                dict(text="Selection 2", x=0.45, xref="paper", y=1.1,
                                     yref="paper", showarrow=False)
            ])
        fig.update_xaxes(categoryorder= 'array', categoryarray= vcnts.index)
        fig.show()
        
    return

In [ ]:
build_graph(Questions['Q1'],Roles,'Age by Position')

In [ ]:
build_graph(Questions['Q12'],Roles,'Hardware by position')